In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
8164,8165,7425660,1298848.0,4/20/2012 8:00:00 AM,149,LAKE SHORE BLVD W,ELLIS AVE,NaN,Major Arterial,Toronto and East York,...,NaN,NaN,NaN,85,South Parkdale,85,South Parkdale (85),D11,623930.5737,4.832663e+06
2940,2941,4270317,1014428.0,12/27/2007 10:00:00 AM,3,VICTORIA PARK AVE,BIGGIN CRT,NaN,Major Arterial,North York,...,NaN,NaN,NaN,43,Victoria Village,43,Victoria Village (43),D33,636562.8143,4.843153e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['MANOEUVER', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 18)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158
17128,6/5/2021 8:00:00 AM,851,PARLIAMENT ST,RICHMOND ST E,Toronto and East York,43.654486,-79.363986,Clear,Daylight,Dry,Non-Fatal Injury,Angle,Major,NaN,Pick Up Truck,Driving Properly,73,Moss Park
5250,11/9/2009 10:00:00 AM,1255,QUEEN Stre W,ST PATRICK Stre,Toronto and East York,43.650445,-79.388690,Clear,Daylight,Dry,Non-Fatal Injury,Pedestrian Collisions,Major,NaN,Other,NaN,78,Kensington-Chinatown


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,...,INJURY,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH
15928,2019-09-08 08:00:00,1515,STEELES AVE E,MOUNTCASTLE GT,Scarborough,43.820498,-79.320415,Clear,Daylight,Dry,...,Major,NaN,NaN,NaN,116,Steeles,Sunday,15,2019,9
6519,2010-11-17 10:00:00,1853,FINCH Ave W,VIRGILWOOD Dr,North York,43.772845,-79.448190,Clear,Daylight,Dry,...,Major,NaN,Other,NaN,35,Westminster-Branson,Wednesday,18,2010,11
16915,2021-02-04 10:00:00,1535,DOWNSVIEW PARK BLVD,KEELE ST,North York,43.736876,-79.484515,Clear,"Daylight, artificial",Dry,...,NaN,NaN,"Automobile, Station Wagon",Failed to Yield Right of Way,155,Downsview,Thursday,15,2021,2


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Twilight',
    'Dawn, artificial': 'Twilight',
    'Dawn': 'Twilight',
    'Dusk': 'Twilight',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    10936
Dark         7298
Twilight      719
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'SnowIce',
    'Packed Snow': 'SnowIce',
    'Slush': 'SnowIce',
    'Ice': 'SnowIce',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry        15389
Wet         3141
SnowIce      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,...,FATAL_NO,VEHTYPE,DRIVACT,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH,SEVERE_COLLISION
15186,2018-11-26,EDWARD ST,BAY ST,Toronto and East York,43.656603,-79.382831,Rain,Dark,Wet,Non-Fatal Injury,...,0.0,NaN,NaN,170,Yonge-Bay Corridor,Monday,16,2018,11,0
15440,2019-03-18,LESLIE ST,FINCH AVE E,North York,43.789939,-79.368030,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,"Automobile, Station Wagon",Driving Properly,47,Don Valley Village,Monday,15,2019,3,0
7007,2011-05-23,ISLINGTON AVE,ALBION RD,Etobicoke York,43.737245,-79.565290,Clear,Daylight,Dry,Non-Fatal Injury,...,0.0,"Automobile, Station Wagon",Driving Properly,3,Thistletown-Beaumond Heights,Monday,13,2011,5,0
8838,2012-10-18,LAKE SHORE Blvd W,BATHURST St,Toronto and East York,43.636473,-79.399638,Rain,Dark,Wet,Non-Fatal Injury,...,0.0,Other,NaN,165,Harbourfront-CityPlace,Thursday,21,2012,10,1
5865,2010-05-23,187 TEDDINGTON PARK AVE,NaN,North York,43.734745,-79.395090,Clear,Dark,Dry,Non-Fatal Injury,...,0.0,Other,NaN,105,Lawrence Park North,Sunday,2,2010,5,0


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 8:
        return 'LateNight'
    elif 8 <= hour < 14:
        return 'MorningAfternoon'
    elif 14 <= hour < 19:
        return 'Evening'
    else:
        return 'Night'

In [35]:
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [36]:
def categorize_vehicle(v):
    if v in ["Automobile, Station Wagon", "Passenger Van", "Taxi", "Other"]:
        return "Car"
    elif v in [
        "Pick Up Truck", "Truck – Open", "Truck – Closed (Blazer, etc)",
        "Truck – Dump", "Truck–Tractor", "Delivery Van",
        "Tow Truck", "Truck (other)", "Truck – Car Carrier",
        "Construction Equipment"
    ]:
        return "Truck"
    elif v in [
        "Municipal Transit Bus (TTC)", "Bus (Other) (Go Bus, Gray Coa)",
        "Intercity Bus", "School Bus"
    ]:
        return "Bus"
    elif v in ["Motorcycle", "Bicycle", "Moped", "Rickshaw"]:
        return "Two Wheeler"
    elif v in ["Police Vehicle", "Fire Vehicle", "Ambulance", "Other Emergency Vehicle"]:
        return "Emergency"
    else:
        return "Car"

In [37]:
def categorize_drivact(a):
    if a == "DrivingProperly":
        return "Driver Proper"
    elif a in ["Failed to Yield Right of Way", "Disobeyed Traffic Control", "Wrong Way on One Way Road"]:
        return "Right-of-Way/Traffic Violation"
    elif a in ["Exceeding Speed Limit", "Speed too Fast For Condition", "Speed too Slow"]:
        return "Speed-Related"
    elif a in ["Lost control", "Improper Turn", "Following too Close", "Improper Lane Change", "Improper Passing"]:
        return "Control/Maneuver Error"
    else:
        return "Other/Unknown"


In [38]:
def categorize_impact(impact):
    if impact in ["Pedestrian Collisions", "Cyclist Collisions"]:
        return "Pedestrian/Cyclist"
    elif impact == "Rear End":
        return "Rear End"
    elif impact in ["Sideswipe", "Approaching"]:
        return "Sideswipe/Approaching"
    elif impact in ["Angle", "Turning Movement"]:
        return "T-Bone"
    elif impact in ["SMV Other", "SMV Unattended Vehicle"]:
        return "Slow/Parked"
    else:
        return "Other/Unknown"

In [39]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)
df['SEASON'] = df['MONTH'].apply(categorize_season)
df["VEHICLE_TYPE"] = df["VEHTYPE"].apply(categorize_vehicle)
df["DRIVER_ACTION"] = df["DRIVACT"].apply(categorize_drivact)
df["IMPACT_TYPE"] = df["IMPACTYPE"].apply(categorize_impact)

In [40]:
df = df[df["IMPACT_TYPE"] != "Other/Unknown"]

In [41]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 1), 'ROAD_CONDITION'] = 'Wet'
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 0), 'ROAD_CONDITION'] = 'Dry'

In [42]:
df['LIGHT'] = df['LIGHT'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 1), 'LIGHT'] = 'Dark'
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 0), 'LIGHT'] = 'Daylight'

In [43]:
df['VISIBILITY'] = df['VISIBILITY'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 1), 'VISIBILITY'] = 'Rain'
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 0), 'VISIBILITY'] = 'Clear'

In [44]:
df.shape

(18726, 27)

In [45]:
df.to_csv('cleaned_ksi_collisions.csv', index=False)